In [1]:
# Suppress warning messages
import warnings
warnings.filterwarnings('ignore')

# Standard libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import json
import itertools
import glob
import sys
import requests
import random
import pickle
import joblib
import imageio
import skimage as sk
import skimage.transform as sktr
import skimage.io as skio
import PIL
from pathlib import Path
from PIL import ImageFont, Image

# Machine learning libraries
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_predict, GridSearchCV
from skimage.transform import resize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Conv2D, Dense, Flatten, MaxPooling2D, AveragePooling2D, Reshape
from keras.optimizers import Adam, SGD
from keras.losses import SparseCategoricalCrossentropy, MeanSquaredError
from keras.callbacks import EarlyStopping
# Suppress Tensorflow warnings and errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import im
from dataset import read_image as load_data

In [3]:
path = '../models/ArchLeNet5.h5'
old_model = load_model(path)
print(old_model.summary())
print(old_model.layers[-1].name)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Reshape (Reshape)           (None, 256, 256, 1)       0         
                                                                 
 Conv2D_1 (Conv2D)           (None, 256, 256, 6)       156       
                                                                 
 AvgPool2D_1 (AveragePooling  (None, 128, 128, 6)      0         
 2D)                                                             
                                                                 
 Conv2D_2 (Conv2D)           (None, 124, 124, 16)      2416      
                                                                 
 AvgPool2D_2 (AveragePooling  (None, 62, 62, 16)       0         
 2D)                                                             
                                                                 
 Flatten (Flatten)           (None, 61504)             0

In [4]:
img_path = 'test.jpg'
img = np.array(sktr.resize(im.gray(im.read(img_path)),(1,256,256,1)))
styles = ['Achaemenid architecture','American craftsman style','American Foursquare architecture','Ancient Egyptian architecture','Art Deco architecture',
    'Art Nouveau architecture','Baroque architecture','Bauhaus architecture','Beaux-Arts architecture','Byzantine architecture',
    'Chicago school architecture','Colonial architecture','Deconstructivism','Edwardian architecture','Georgian architecture',
    'Gothic architecture','Greek Revival architecture','International style','Novelty architecture','Palladian architecture',
    'Postmodern architecture','Queen Anne architecture','Romanesque architecture','Russian Revival architecture','Tudor Revival architecture']
ans = old_model.predict(img)
print(ans)
print(styles[np.argmax(ans)])

1/1 [==============================] - 0s 103ms/step
[[0.55704504 0.00309595 0.01381878 0.09544117 0.0332514  0.00622243
  0.02278186 0.00656865 0.00322705 0.18028963 0.00156283 0.00170586
  0.00114412 0.00133506 0.00135451 0.00249074 0.00153237 0.00406687
  0.00194032 0.00300134 0.00191499 0.00253556 0.00225248 0.05068374
  0.00073728]]
Achaemenid architecture


In [5]:
conv_model = Model(inputs=old_model.input, outputs=old_model.output)

new_model = Sequential()
new_model.add(old_model)
new_model.add(Dense(20,activation='relu'))
new_model.add(Dense(8,activation='relu'))
new_model.add(Dense(1,activation='linear'))

print(new_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 25)                7395461   
                                                                 
 dense (Dense)               (None, 20)                520       
                                                                 
 dense_1 (Dense)             (None, 8)                 168       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 7,396,158
Trainable params: 7,396,158
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
train, test = load_data(gray_img=True)
train_x, train_y = zip(*train)
test_x, test_y = zip(*test)

~1500
1500~1800
1800~1970
1970~


In [7]:

train_x1, train_y1, test_x1, test_y1 = np.array(train_x), np.array(train_y), np.array(test_x), np.array(test_y)
print(train_x1)
print(train_y1)

[[[0.85014782 0.85212114 0.85310179 ... 0.42854439 0.4236493  0.28462128]
  [0.87338964 0.87778597 0.87982949 ... 0.32516386 0.30569409 0.27508042]
  [0.8808495  0.88190774 0.88916228 ... 0.27739231 0.27093325 0.31592226]
  ...
  [0.4611994  0.46085128 0.4222378  ... 0.24548217 0.25771457 0.31317069]
  [0.42025116 0.32323787 0.37748531 ... 0.33779624 0.35067901 0.35454237]
  [0.44908843 0.36600658 0.37977862 ... 0.36169399 0.33090685 0.36934833]]

 [[0.85062519 0.85112759 0.85172518 ... 0.88762231 0.88471975 0.88187695]
  [0.84921676 0.85359477 0.85378346 ... 0.88496732 0.88496732 0.88496732]
  [0.84983907 0.85211676 0.85415645 ... 0.88496732 0.88496732 0.88496732]
  ...
  [0.0949407  0.08322124 0.06652992 ... 0.39658219 0.44420717 0.4313984 ]
  [0.07084618 0.07648942 0.06304732 ... 0.41894021 0.46278339 0.39328733]
  [0.06337125 0.08292452 0.07309905 ... 0.45314351 0.41886609 0.36552878]]

 [[0.88209692 0.88226143 0.87704871 ... 0.91171893 0.91494116 0.91563849]
  [0.88578865 0.879741

In [8]:
new_model.layers[0].trainable = False

for layer in new_model.layers:
    print("{0}:\t{1}".format(layer.trainable, layer.name))


False:	sequential
True:	dense
True:	dense_1
True:	dense_2


In [14]:
optimizer = Adam(learning_rate=0.01)
loss = MeanSquaredError()

new_model.compile(optimizer=optimizer,loss=loss)


In [15]:
# callbacks = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 2)
new_model_history = new_model.fit(train_x1,train_y1,validation_data=(test_x1,test_y1),epochs=15,steps_per_epoch=50,shuffle=True)

Epoch 1/15
50/50 [==============================] - 6s 104ms/step - loss: 647562.3125 - val_loss: 672467.6875
Epoch 2/15
50/50 [==============================] - 5s 95ms/step - loss: 659000.5625 - val_loss: 671877.4375
Epoch 3/15
50/50 [==============================] - 5s 95ms/step - loss: 693453.6250 - val_loss: 674232.8125
Epoch 4/15
50/50 [==============================] - 5s 100ms/step - loss: 625224.8750 - val_loss: 675993.1250
Epoch 5/15
50/50 [==============================] - 5s 102ms/step - loss: 684548.0000 - val_loss: 673271.5000
Epoch 6/15
50/50 [==============================] - 5s 105ms/step - loss: 606562.5000 - val_loss: 674731.6250
Epoch 7/15
50/50 [==============================] - 5s 98ms/step - loss: 640589.6875 - val_loss: 674891.8125
Epoch 8/15
50/50 [==============================] - 5s 102ms/step - loss: 645020.5625 - val_loss: 674773.5000
Epoch 9/15
50/50 [==============================] - 5s 104ms/step - loss: 641659.7500 - val_loss: 677713.7500
Epoch 10/15
5

In [12]:
img_path = 'test.jpg'
img = np.array(sktr.resize(im.gray(im.read(img_path)),(1,256,256,1)))
print(new_model.predict(img))

1/1 [==============================] - 0s 83ms/step
[[1329.711]]


In [13]:
err = 0
for img, era in zip(test_x,test_y):
    img = np.array(sktr.resize(img,(1,256,256,1)))
    pred = new_model.predict(img)
    err += abs(pred-era)
    print("pred:%d, true:%d, diff:%d" %(pred,era,abs(pred-era)))
print(err/len(test_x))

1/1 [==============================] - 0s 30ms/step
pred:1353, true:1712, diff:358
1/1 [==============================] - 0s 22ms/step
pred:1427, true:1953, diff:525
1/1 [==============================] - 0s 22ms/step
pred:1411, true:-430, diff:1841
1/1 [==============================] - 0s 23ms/step
pred:1443, true:1887, diff:443
1/1 [==============================] - 0s 21ms/step
pred:1413, true:1926, diff:512
1/1 [==============================] - 0s 25ms/step
pred:1422, true:1699, diff:276
1/1 [==============================] - 0s 21ms/step
pred:1488, true:1965, diff:476
1/1 [==============================] - 0s 23ms/step
pred:1422, true:1826, diff:403
1/1 [==============================] - 0s 24ms/step
pred:1419, true:1933, diff:513
1/1 [==============================] - 0s 23ms/step
pred:1414, true:1912, diff:497
1/1 [==============================] - 0s 22ms/step
pred:1339, true:1722, diff:382
1/1 [==============================] - 0s 23ms/step
pred:1315, true:1940, diff:624
1/1